In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [2]:
c=requests.get('https://imoveis.mercadolivre.com.br/aluguel/').content

In [4]:
# Parameters
zonas = ['centro', 'zona-oeste', 'zona-norte', 'zona-sul']
URL='https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/rio-de-janeiro-{}/{}'
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', 
           '_Desde_241', '_Desde_289']

In [5]:
# Functions
def scrap(url):
    c=requests.get(url).content
    soup=BeautifulSoup(c)
    aluguel_tags = soup.find_all('span', {'class':'price-tag-fraction'})
    alugueis_lista = [p.text for p in aluguel_tags]
    enderecos_tags = soup.find_all('span', {'class':'ui-search-item__group__element'})
    enderecos_lista=[end.text for end in enderecos_tags]
    enderecos_lista=enderecos_lista[1::2]
    area_quartos_tags = soup.find_all('ul', {'class':'ui-search-card-attributes'})
    area_quartos_lista=[aq.text for aq in area_quartos_tags]
    df=convert_df(zona, area_quartos_lista, enderecos_lista, alugueis_lista)
    return df

def convert_df(zona, area_quartos_lista, enderecos_lista, alugueis_lista):
    zonas_lista = [zona]*len(alugueis_lista)
    data = {'zona': zonas_lista,
            'area_quartos': area_quartos_lista,
            'enderecos_lista':enderecos_lista,
            'aluguel': alugueis_lista}
    df = pd.DataFrame(data)
    return df

In [8]:
# Execution
df_lista = []
for zona in zonas:
  for pg in paginas:
    url=URL.format(zona, pg)
    print('Coletando URL:', url)
    try:
      df=scrap(url)
    except Exception as e:
      print('Erro ao coletar URL:', url)
      print(e)
    df_lista.append(df)
    sleep(1)
df_final = pd.concat(df_lista)

Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/rio-de-janeiro-centro/
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/rio-de-janeiro-centro/_Desde_49
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/rio-de-janeiro-centro/_Desde_97
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/rio-de-janeiro-centro/_Desde_145
Erro ao coletar URL: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/rio-de-janeiro-centro/_Desde_145
arrays must all be same length
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/rio-de-janeiro-centro/_Desde_193
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/rio-de-janeiro-centro/_Desde_241
Erro ao coletar URL: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/rio-de-janeiro-centro/_Desde_241
arrays must all be same length
Coletando URL: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/rio-de-j

In [9]:
df_final.head()

,zona,area_quartos,enderecos_lista,aluguel
0,centro,50 m² construídos1 quarto,"Rua Senador Dantas, Centro, Rio De Janeiro Cen...",970
1,centro,28 m² construídos1 quarto,"Rua Das Marrecas, Centro, Rio De Janeiro Centr...",1.224
2,centro,38 m² construídos1 quarto,"Rua De Santana, Centro, Rio De Janeiro Centro,...",1.000
3,centro,62 m² construídos2 quartos,"Rua Riachuelo, Centro, Rio De Janeiro Centro, ...",2.200
4,centro,48 m² construídos2 quartos,"Rua João Neves Da Fontoura, Centro, Rio De Jan...",1.001


In [10]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1311 entries, 0 to 47
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   zona             1311 non-null   object
 1   area_quartos     1311 non-null   object
 2   enderecos_lista  1311 non-null   object
 3   aluguel          1311 non-null   object
dtypes: object(4)
memory usage: 51.2+ KB
